In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image as keras_image
from tensorflow.keras.applications.inception_v3 import preprocess_input
import cv2
import numpy as np

# Define image transformations (same as used during training)
def preprocess_image(img):
    img = cv2.resize(img, (224, 224))  # Resize to the size your model expects
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
    img = keras_image.img_to_array(img)  # Convert image to array
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    img = preprocess_input(img)  # Preprocess the image (e.g., normalization)
    return img

# Load the trained model
model = tf.keras.models.load_model('/content/model (1).keras')

# Function to process the video and classify each frame
def predict_video(video_path, model):
    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    real_count = 0
    spoof_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1

        # Preprocess the frame
        image = preprocess_image(frame)

        # Make prediction
        predictions = model.predict(image)
        predicted = np.argmax(predictions)

        if predicted == 0:
            real_count += 1
        else:
            spoof_count += 1

    cap.release()

    # Final decision based on majority vote across all frames
    if real_count > spoof_count:
        print(f"Result: Real video ({real_count} real frames, {spoof_count} deepfake frames)")
        return "Real"
    else:
        print(f"Result: Spoof video ({real_count} real frames, {spoof_count} deepfake frames)")
        return "Fake"

# Test the video
video_path = "speech_happy__JZUXXFRB.mp4"
result = predict_video(video_path, model)
